Virtual Enviromnemt Snapshot

In [1]:
!pip list

Package                                  Version
---------------------------------------- --------------
aiofiles                                 24.1.0
aiohappyeyeballs                         2.6.1
aiohttp                                  3.12.8
aiosignal                                1.3.2
annotated-types                          0.7.0
anyio                                    4.9.0
argon2-cffi                              25.1.0
argon2-cffi-bindings                     21.2.0
arrow                                    1.3.0
asgiref                                  3.8.1
asttokens                                3.0.0
async-lru                                2.0.5
async-timeout                            4.0.3
attrs                                    25.3.0
babel                                    2.17.0
backcall                                 0.2.0
backoff                                  2.2.1
bcrypt                                   4.3.0
beautifulsoup4                           4.

Things that you will need 

1. Ollama (I am using llama3.2 from ollama you could use any other LLM from ollama you like )
2. Langchain Community
3. chroma db for vector database
4. Unstructured for documents
5. For embeddings you could use OpenAi embeddings 

In [ ]:
!ollama list 

NAME                 ID              SIZE      MODIFIED    
llama3.2:latest      a80c4f17acd5    2.0 GB    4 weeks ago    
openhermes:latest    95477a2659b7    4.1 GB    4 weeks ago    
qwen:0.5b            b5dc5e784f2a    394 MB    4 weeks ago    


In [ ]:
!pip install -U langchain-community

In [ ]:
!pip install langchain_openai #optional

In [ ]:
!pip install chromadb

In [ ]:
!pip install unstructured

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install "unstructured[md]"

In [ ]:
!pip install ollama

In [5]:
# Chroma db to Store data that we have processed
from langchain.vectorstores.chroma import Chroma
# os is to access files from directory
import os
import shutil
# from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
# import HuggingFaceEmbeddings to create embeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

In [6]:
DATA_PATH = 'Data'
new_chunks = []
CHROMA_PATH = 'CHROMA'

In [7]:
from langchain.document_loaders.pdf import PyPDFDirectoryLoader

def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="*.md")
    documents = loader.load()
    return documents

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document

def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=100,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    document = chunks[0]
    print(document.page_content)
    print(document.metadata)
    return chunks

Using Hugging Face embedding function

In [9]:
def save_to_chroma(chunks: list[Document]):
    # Clear out the database first.
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)

    # Create a new DB from the documents.
    db = Chroma.from_documents(
        chunks, HuggingFaceEmbeddings(), persist_directory=CHROMA_PATH
    )

    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")



Lets use Ollama Embedding function

In [ ]:

# def add_to_chroma(chunks:list[Document]):
#     db = Chroma(
#         persist_directory=CHROMA_PATH,
#         embedding_function=HuggingFaceEmbeddings()

#     )


In [10]:

def generate_data_store():
    documents = load_documents()
    chunks = split_text(documents)
    save_to_chroma(chunks)


In [11]:
def main():
    generate_data_store()
if __name__ == "__main__":
    main()

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


Split 1 documents into 818 chunks.
The Project Gutenberg eBook of Alice's Adventures in Wonderland
{'source': 'Data\\alice_in_wonderland.md', 'start_index': 0}


C:\Users\Com\AppData\Local\Temp\ipykernel_4108\2192508030.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  chunks, HuggingFaceEmbeddings(), persist_directory=CHROMA_PATH
C:\Users\Com\AppData\Local\Temp\ipykernel_4108\2192508030.py:8: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  chunks, HuggingFaceEmbeddings(), persist_directory=CHROMA_PATH
c:\Users\Com\anaconda3\envs\RagEnv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipy

Saved 818 chunks to CHROMA.


C:\Users\Com\AppData\Local\Temp\ipykernel_4108\2192508030.py:11: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [12]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

In [13]:
from langchain.prompts import ChatPromptTemplate

In [14]:
import ollama

In [15]:
def query_rag(query_text: str):
    # Prepare the DB.
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=HuggingFaceEmbeddings())

    # Search the DB.
    results = db.similarity_search_with_score(query_text, k=5)
    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    response  = ollama.chat(
        model = "llama3.2",
        messages = [
            {'role':'user', "content":prompt}
            ]
    )
    return response['message']['content']


In [17]:
query_rag("is alice strong")

C:\Users\Com\AppData\Local\Temp\ipykernel_4108\2154296689.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  db = Chroma(persist_directory=CHROMA_PATH, embedding_function=HuggingFaceEmbeddings())


'Based on the context, it can be inferred that Alice is not particularly strong. She mentions feeling "uncomfortable" while shrunk in size and appears to be worried about her future prospects of learning new things and never aging, suggesting a vulnerability or sensitivity to her situation rather than any physical strength.'